In [13]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from rouge_score import rouge_scorer

In [14]:
# load tokenizer and model, create trainer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)
# similarities = {}
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=False)

In [9]:
one_instruct = {
        "instruction": "Create a witty and funny headline for a new product.",
        "input": "",
        "output": "\"Say Cheese!\" - The Revolutionary Camera That Captures Smiles Instead of Photos.",}

# tokenize input
tokenized_texts = tokenizer(one_instruct["instruction"], one_instruct["input"], one_instruct["output"], truncation=True, padding=True)
print(tokenized_texts)

{'input_ids': [0, 44758, 10, 33228, 8, 6269, 8707, 13, 10, 92, 1152, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 113, 34673, 25436, 2901, 111, 20, 22110, 11149, 280, 13007, 4123, 4966, 4755, 2978, 9, 10878, 4, 2]}


In [18]:
all_insturcts = [one_instruct["instruction"]] + [one_instruct["output"]]

In [15]:
all_instruction_tokens = [scorer._tokenizer.tokenize(inst) for inst in all_instructions]

AttributeError: 'dict' object has no attribute 'lower'

In [10]:
# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [11]:
pred_dataset = SimpleDataset(tokenized_texts)

In [12]:
# Run predictions
predictions = trainer.predict(pred_dataset)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)